In [ ]:
import ee
import os

try:
    ee.Initialize(project='parth-362005')
except Exception:
    ee.Authenticate()
    ee.Initialize(project='parth-362005')


aoi = ee.Geometry.Polygon([
    [[90.0, 24.0], [90.5, 24.0], [90.5, 24.5], [90.0, 24.5], [90.0, 24.0]]
])

# Define time period 
start_date = "2022-01-01"
end_date = "2023-12-31"

# Digital Elevation Model
dem = ee.Image("USGS/SRTMGL1_003").clip(aoi)

# Calculate slope and aspect (important topographic features for flood modeling)
slope = ee.Terrain.slope(dem).clip(aoi)
aspect = ee.Terrain.aspect(dem).clip(aoi)

# Rainfall data - both mean and maximum (for extreme events)
rainfall_mean = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
    .filterDate(start_date, end_date) \
    .select("total_precipitation_sum") \
    .mean().clip(aoi)

rainfall_max = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
    .filterDate(start_date, end_date) \
    .select("total_precipitation_sum") \
    .max().clip(aoi)

# Soil moisture
soil_moisture = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/006") \
    .filterDate(start_date, end_date) \
    .select("soil_moisture") \
    .mean().clip(aoi)

# Land cover
land_cover = ee.ImageCollection("MODIS/061/MCD12Q1") \
    .filterDate(start_date, end_date) \
    .select("LC_Type1") \
    .mode().clip(aoi)

# Sentinel-1 for flood detection (VV and VH polarization)
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .mean().clip(aoi)

# NDVI for vegetation cover from Sentinel-2
s2 = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median().clip(aoi)

ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Water occurrence (from JRC Global Surface Water)
water_occurrence = ee.Image("JRC/GSW1_3/GlobalSurfaceWater").select('occurrence').clip(aoi)

# Population density for vulnerability assessment
population = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Density") \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
    .mean().clip(aoi)

# Export all layers to Google Drive
export_tasks = {
    "DEM": dem,
    "Slope": slope,
    "Aspect": aspect,
    "Rainfall_Mean": rainfall_mean,
    "Rainfall_Max": rainfall_max,
    "Soil_Moisture": soil_moisture,
    "Land_Cover": land_cover,
    "Sentinel1_VV": sentinel1.select('VV'),
    "NDVI": ndvi,
    "Water_Occurrence": water_occurrence,
    "Population": population
}

# Start export tasks
for name, image in export_tasks.items():
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f"{name}_Flood_Risk",
        folder="FloodRiskData",
        fileNamePrefix=name.lower(),
        scale=30,
        region=aoi,
        fileFormat="GeoTIFF",
        maxPixels=1e13  # Increased max pixels for larger areas
    )
    task.start()

print("✅ Enhanced data export started successfully!")


In [ ]:
import rasterio
import numpy as np
from sklearn.model_selection import train_test_split

def load_raster(file_path):
    try:
        with rasterio.open(file_path) as src:
            data = src.read(1)
            data = data.astype(np.float32)
            # Handle no data values
            if src.nodata is not None:
                data[data == src.nodata] = np.nan
            # Replace NaNs with the mean of valid values
            if np.any(np.isnan(data)):
                valid_data = data[~np.isnan(data)]
                if len(valid_data) > 0:
                    data = np.nan_to_num(data, nan=np.mean(valid_data))
                else:
                    data = np.nan_to_num(data)
            return data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Load all raster datasets
raster_files = {
    "dem": "dem.tif",
    "slope": "slope.tif",
    "aspect": "aspect.tif",
    "rainfall_mean": "rainfall_mean.tif",
    "rainfall_max": "rainfall_max.tif",
    "soil_moisture": "soil_moisture.tif",
    "land_cover": "land_cover.tif",
    "sentinel1_vv": "sentinel1_vv.tif",
    "ndvi": "ndvi.tif",
    "water_occurrence": "water_occurrence.tif",
    "population": "population.tif",
    "flood_mask": "flood_mask.tif"  # Historical flood mask if available
}

# Load all available datasets
datasets = {}
for name, file_path in raster_files.items():
    if os.path.exists(file_path):
        datasets[name] = load_raster(file_path)
        print(f"Loaded {name}")
    else:
        print(f"Warning: {file_path} not found, skipping.")

# Find minimum dimensions across all datasets
shapes = [dataset.shape for dataset in datasets.values() if dataset is not None]
min_height = min(shape[0] for shape in shapes)
min_width = min(shape[1] for shape in shapes)

# Crop all datasets to the same dimensions
for name in datasets:
    if datasets[name] is not None:
        datasets[name] = datasets[name][:min_height, :min_width]

# Normalize continuous data (not categorical data)
for name in datasets:
    if name != "land_cover" and name != "flood_mask" and datasets[name] is not None:
        min_val = np.min(datasets[name])
        max_val = np.max(datasets[name])
        if max_val > min_val:  # Avoid division by zero
            datasets[name] = (datasets[name] - min_val) / (max_val - min_val)

# Create feature stack (X) and target (y)
feature_names = [name for name in datasets if name != "flood_mask"]
X = np.stack([datasets[name] for name in feature_names if datasets[name] is not None], axis=-1)

# If we have a flood mask, use it; otherwise use water occurrence as proxy
if "flood_mask" in datasets and datasets["flood_mask"] is not None:
    y = (datasets["flood_mask"] > 0).astype(np.uint8)
else:
    # Use water occurrence as proxy for flood risk
    y = (datasets["water_occurrence"] > 25).astype(np.uint8)

y = y.reshape(y.shape[0], y.shape[1], 1)

# Extract patches with improved patch extraction (with overlap for better coverage)
patch_size = 256
stride = 128

# Function to extract patches
def extract_patches(image, patch_size, stride):
    patches = []
    locations = []  # Store patch coordinates for visualization later

    max_row = max(0, image.shape[0] - patch_size + 1)
    max_col = max(0, image.shape[1] - patch_size + 1)

    for i in range(0, max_row, stride):
        for j in range(0, max_col, stride):
            patch = image[i:i + patch_size, j:j + patch_size]
            # Skip mostly empty patches (e.g., patches with very few flood pixels)
            if np.mean(patch) > 0.0001:  # Adjust threshold as needed
                patches.append(patch)
                locations.append((i, j))
    while len(patches) == 0 and patch_size >= 32:  # Minimum patch size of 32
        print(f"No patches found with patch_size={patch_size}, stride={stride}. Reducing size...")
        patch_size //= 2  # Reduce patch size by half
        stride //= 2  # Reduce stride by half

        max_row = max(0, image.shape[0] - patch_size + 1)
        max_col = max(0, image.shape[1] - patch_size + 1)

        patches = []  # Reset patches list
        locations = []  # Reset locations list

        for i in range(0, max_row, stride):
            for j in range(0, max_col, stride):
                patch = image[i:i + patch_size, j:j + patch_size]
                if np.mean(patch) > 0.0001:  # Adjust threshold as needed
                    patches.append(patch)
                    locations.append((i, j))

    if len(patches) == 0:
        raise ValueError("Patch size too small. No patches can be created.")

    return np.array(patches), locations

# Extract patches
X_patches, X_locations = extract_patches(X, patch_size, stride)
y_patches, _ = extract_patches(y, patch_size, stride)

# Print dataset information
print(f"Feature dimensions: {X.shape}, with {X.shape[-1]} features")
print(f"Target dimensions: {y.shape}")
print(f"Extracted {len(X_patches)} patches of size {patch_size}x{patch_size}")

# Data augmentation for model robustness
def augment_data(X_data, y_data):
    X_augmented = []
    y_augmented = []

    for i in range(len(X_data)):
        # Original
        X_augmented.append(X_data[i])
        y_augmented.append(y_data[i])

        # Flip horizontally
        X_augmented.append(np.flip(X_data[i], axis=1))
        y_augmented.append(np.flip(y_data[i], axis=1))

        # Flip vertically
        X_augmented.append(np.flip(X_data[i], axis=0))
        y_augmented.append(np.flip(y_data[i], axis=0))

        # Rotate 90 degrees
        X_augmented.append(np.rot90(X_data[i], k=1))
        y_augmented.append(np.rot90(y_data[i], k=1))

    return np.array(X_augmented), np.array(y_augmented)

X_aug, y_aug = augment_data(X_patches, y_patches)
print(f"Dataset size after augmentation: {len(X_aug)} samples")

# Split Data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_aug, y_aug, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import pandas as pd
import seaborn as sns

input_shape = (patch_size, patch_size, X.shape[-1])
print(f"Model input shape: {input_shape}")

# Improved UNet model with batch normalization and dropout
inputs = Input(input_shape)

# Encoder path with batch normalization
c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
c1 = BatchNormalization()(c1)
c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
c1 = BatchNormalization()(c1)
p1 = MaxPooling2D((2, 2))(c1)
p1 = Dropout(0.1)(p1)

c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
c2 = BatchNormalization()(c2)
c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
c2 = BatchNormalization()(c2)
p2 = MaxPooling2D((2, 2))(c2)
p2 = Dropout(0.2)(p2)

c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
c3 = BatchNormalization()(c3)
c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
c3 = BatchNormalization()(c3)
p3 = MaxPooling2D((2, 2))(c3)
p3 = Dropout(0.3)(p3)

c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
c4 = BatchNormalization()(c4)
c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
c4 = BatchNormalization()(c4)
p4 = MaxPooling2D((2, 2))(c4)
p4 = Dropout(0.4)(p4)

# Middle
c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
c5 = BatchNormalization()(c5)
c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
c5 = BatchNormalization()(c5)
c5 = Dropout(0.5)(c5)

# Decoder path with skip connections
u6 = UpSampling2D((2, 2))(c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
c6 = BatchNormalization()(c6)
c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
c6 = BatchNormalization()(c6)
c6 = Dropout(0.4)(c6)

u7 = UpSampling2D((2, 2))(c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
c7 = BatchNormalization()(c7)
c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
c7 = BatchNormalization()(c7)
c7 = Dropout(0.3)(c7)

u8 = UpSampling2D((2, 2))(c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
c8 = BatchNormalization()(c8)
c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
c8 = BatchNormalization()(c8)
c8 = Dropout(0.2)(c8)

u9 = UpSampling2D((2, 2))(c8)
u9 = concatenate([u9, c1])
c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
c9 = BatchNormalization()(c9)
c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
c9 = BatchNormalization()(c9)
c9 = Dropout(0.1)(c9)

# Output layer
outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

# Create the model
model = Model(inputs=[inputs], outputs=[outputs])

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Compile with binary cross-entropy + dice loss
model.compile(optimizer='adam',
              loss=lambda y_true, y_pred: tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred),
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Print model summary
model.summary()

# Set up callbacks for better training
checkpoint = ModelCheckpoint('flood_risk_model_best.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min',
                             verbose=1)

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=5,
                              min_lr=1e-6,
                              verbose=1)

callbacks = [checkpoint, early_stopping, reduce_lr]

# Train the model with more epochs and callbacks
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,  # More epochs with early stopping
    batch_size=16,  # Smaller batch size often works better
    callbacks=callbacks,
    verbose=1
)

In [ ]:
model.save('flood_risk_model_final.h5')

# Evaluate on test set
test_results = model.evaluate(X_test, y_test, verbose=1)
print(f"Test loss: {test_results[0]:.4f}")
print(f"Test accuracy: {test_results[1]:.4f}")
print(f"Test recall: {test_results[2]:.4f}")
print(f"Test precision: {test_results[3]:.4f}")

# Make predictions on test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(np.uint8)

# Calculate confusion matrix
y_true_flat = y_test.reshape(-1)
y_pred_flat = y_pred_binary.reshape(-1)
cm = confusion_matrix(y_true_flat, y_pred_flat)

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_true_flat, y_pred_flat, average='binary')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Plot training history
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

# Visualize a few examples
plt.figure(figsize=(15, 10))
num_samples = min(5, len(X_test))

for i in range(num_samples):
    # Original image (show DEM as background)
    plt.subplot(3, num_samples, i+1)
    plt.imshow(X_test[i][:,:,0], cmap='terrain')
    plt.title(f"Sample {i+1}")
    plt.axis('off')

    # True mask
    plt.subplot(3, num_samples, i+1+num_samples)
    plt.imshow(y_test[i].reshape(patch_size, patch_size), cmap='Blues')
    plt.title(f"True Flood Mask")
    plt.axis('off')

    # Predicted mask
    plt.subplot(3, num_samples, i+1+2*num_samples)
    plt.imshow(y_pred[i].reshape(patch_size, patch_size), cmap='Blues')
    plt.title(f"Predicted Flood Risk")
    plt.axis('off')

plt.tight_layout()
plt.savefig('prediction_examples.png')
plt.close()

# Visualization of confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Flood', 'Flood'],
            yticklabels=['No Flood', 'Flood'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.close()


In [ ]:
def create_full_risk_map(model, X, patch_size, stride):
    # Create an empty risk map
    height, width = X.shape[0], X.shape[1]
    risk_map = np.zeros((height, width))
    count_map = np.zeros((height, width))

    # Generate predictions for patches across the image
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            # Extract patch
            patch = X[i:i + patch_size, j:j + patch_size, :]

            # Make prediction
            patch = patch.reshape(1, patch_size, patch_size, X.shape[-1])
            pred = model.predict(patch)[0].reshape(patch_size, patch_size)

            # Add to risk map and count
            risk_map[i:i + patch_size, j:j + patch_size] += pred
            count_map[i:i + patch_size, j:j + patch_size] += 1

    # Average the overlapping areas
    count_map[count_map == 0] = 1  # Avoid division by zero
    risk_map = risk_map / count_map

    return risk_map

# Create the flood risk map
flood_risk_map = create_full_risk_map(model, X, patch_size, stride)

# Create risk categories
risk_categories = np.zeros_like(flood_risk_map)
risk_categories[(flood_risk_map >= 0.2) & (flood_risk_map < 0.4)] = 1  # Low risk
risk_categories[(flood_risk_map >= 0.4) & (flood_risk_map < 0.6)] = 2  # Medium risk
risk_categories[(flood_risk_map >= 0.6) & (flood_risk_map < 0.8)] = 3  # High risk
risk_categories[flood_risk_map >= 0.8] = 4  # Very high risk

# Save risk map as GeoTIFF
if os.path.exists('dem.tif'):
    with rasterio.open('dem.tif') as src:
        profile = src.profile.copy()
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open('flood_risk_map.tif', 'w', **profile) as dst:
            dst.write(flood_risk_map.astype(np.float32), 1)

        profile.update(dtype=rasterio.uint8, count=1)
        with rasterio.open('flood_risk_categories.tif', 'w', **profile) as dst:
            dst.write(risk_categories.astype(np.uint8), 1)

# Visualize the final risk map
plt.figure(figsize=(12, 10))
plt.subplot(2, 1, 1)
plt.imshow(flood_risk_map, cmap='Blues', vmin=0, vmax=1)
plt.colorbar(label='Flood Risk Probability')
plt.title('Flood Risk Probability Map')

plt.subplot(2, 1, 2)
cmap = plt.cm.get_cmap('viridis', 5)
risk_plot = plt.imshow(risk_categories, cmap=cmap, vmin=0, vmax=4)
categories = ['No Risk', 'Low Risk', 'Medium Risk', 'High Risk', 'Very High Risk']
cbar = plt.colorbar(ticks=range(5))
cbar.set_ticklabels(categories)
plt.title('Flood Risk Categories')

plt.tight_layout()
plt.savefig('flood_risk_map.png')
plt.close()

# Calculate risk statistics
total_area = risk_categories.size
risk_stats = {
    'No Risk': np.sum(risk_categories == 0) / total_area * 100,
    'Low Risk': np.sum(risk_categories == 1) / total_area * 100,
    'Medium Risk': np.sum(risk_categories == 2) / total_area * 100,
    'High Risk': np.sum(risk_categories == 3) / total_area * 100,
    'Very High Risk': np.sum(risk_categories == 4) / total_area * 100
}

# Print risk statistics
print("\nFlood Risk Statistics (% of total area):")
for category, percentage in risk_stats.items():
    print(f"{category}: {percentage:.2f}%")

print("\n✅ Flood risk assessment completed! Risk maps saved as images and GeoTIFF files.")